## Installation Steps

In [ ]:
import os
os.chdir('/content/drive/MyDrive/OSNet')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Total 9809 (delta 0), reused 0 (delta 0), pack-reused 9809
Receiving objects: 100% (9809/9809), 9.54 MiB | 16.64 MiB/s, done.
Resolving deltas: 100% (7263/7263), done.


In [ ]:
cd "/content/drive/MyDrive/OSNet/deep-person-reid"

/content/drive/MyDrive/OSNet/deep-person-reid


In [ ]:
pip install -r requirements.txt

In [ ]:
!pip install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!python setup.py develop

running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/torchreid/metrics/rank_cylib/rank_cy.cpython-37m-x86_64-linux-gnu.so -> torchreid/metrics/rank_cylib
Creating /usr/local/lib/python3.7/dist-packages/torchreid.egg-link (link to .)
Adding torchreid 1.4.0 to easy-install.pth file

Installed /content/drive/MyDrive/OSNet/deep-person-reid
Processing dependencies for torchreid==1.4.0
Searching for isort==4.3.21
Reading https://pypi.org/simple/isort/
Best match: isort 4.3.21
Processing isort-4.3.21-py2.py3-none-any.whl
Installing isort-4.3.21-py2.py3-none-any.whl to /usr/local/lib/python3.7/dist-packages
Adding isort 4.3.21 to easy-install.pth file
Installing

In [ ]:
!python setup.py develop

Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it depends on /usr/local/lib/python3.7/dist-packages/Cython/Includes/libc/string.pxd.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/OSNet/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'torchreid.egg-info/SOURCES.txt'
running build_ext
building 'torchreid.metrics.rank_cylib.rank_cy' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -W

## Adam Optimizer with LR = 0.0003

In [ ]:
import torchreid

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

from torchvision import models
# from torchsummary import summary
# summary(model,(3,256,128))

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003,
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=150,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training
epoch: [1/150][10/404]	time 0.503 (0.589)	data 0.000 (0.073)	eta 9:54:24	loss 6.8726 (6.7731)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/150][20/404]	time 0.503 (0.546)	data 0.000 (0.037)	eta 9:10:59	loss 6.7682 (6.8591)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/150][30/404]	time 0.502 (0.531)	data 0.000 (0.024)	eta 8:56:03	loss 6.7725 (6.7937)	acc 0.0000 (0.5208)	lr 0.000300
epoch: [1/150][40/404]	time 0.504 (0.524)	data 0.000 (0.018)	eta 8:48:31	loss 6.6208 (6.7624)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/150][50/404]	time 0.500 (0.519)	data 0.000 (0.015)	eta 8:43:59	loss 6.6099 (6.7321)	acc 3.1250 (0.4375)	lr 0.000300
epoch: [1/150][60/404]	time 0.502 (0.516)	data 0.000 (0.012)	eta 8:41:04	loss 6.5419 (6.7070)	acc 3.1250 (0.4167)	lr 0.000300
epoch: [1/150][70/404]	time 0.499 (0.514)	data 0.000 (0.011)	eta 8:39:01	loss 6.5360 (6.6837)	acc 3.1250 (0.4911)	lr 0.000300
epoch: [1/150][80/404]	time 0.496 (0.513)	data 0.000 (0.009)	eta 8:37:08	loss 6.5508 (6.6761)	acc 0.

In [ ]:
start_epoch = torchreid.utils.resume_from_checkpoint(
    '/content/drive/MyDrive/OSNet/deep-person-reid/log/resnet50/model/model.pth.tar-130',
    model,
    optimizer
)

engine.run(
    save_dir='log/resnet50',
    max_epoch=150,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    start_epoch = start_epoch
)

Loading checkpoint from "/content/drive/MyDrive/OSNet/deep-person-reid/log/resnet50/model/model.pth.tar-80"
Loaded model weights
Loaded optimizer
Last epoch = 80
Last rank1 = 85.3%
=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [81/150][10/404]	time 0.458 (2.687)	data 0.002 (2.215)	eta 21:06:11	loss 1.0544 (1.0653)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][20/404]	time 0.464 (2.368)	data 0.000 (1.902)	eta 18:35:08	loss 1.0655 (1.0674)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][30/404]	time 0.459 (2.260)	data 0.000 (1.794)	eta 17:44:08	loss 1.0817 (1.0677)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][40/404]	time 0.470 (2.086)	data 0.000 (1.620)	eta 16:21:39	loss 1.1089 (1.0718)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][50/404]	time 0.469 (2.400)	data 0.001 (1.934)	eta 18:49:09	loss 1.0700 (1.0703)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][60/404]	time 0.477 (2.320)	data 0.000 (1.853)	eta 18:10:59	loss 1.0617 (1.0700)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][70/404]	time 0.463 (2.280)	data 0.001 (1.813)	eta 17:52:07	loss 1.0540 (1.0699)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [81/150][80/404]	time 0.469 (2.389)	data 0.001 (1.922)	eta 18:43:01	l

In [ ]:
start_epoch = torchreid.utils.resume_from_checkpoint(
    '/content/drive/MyDrive/OSNet/deep-person-reid/log/resnet50/model/model.pth.tar-130',
    model,
    optimizer
)

engine.run(
    save_dir='log/resnet50',
    max_epoch=150,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    start_epoch = start_epoch
)

Loading checkpoint from "/content/drive/MyDrive/OSNet/deep-person-reid/log/resnet50/model/model.pth.tar-130"
Loaded model weights
Loaded optimizer
Last epoch = 130
Last rank1 = 85.5%
=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [131/150][10/404]	time 0.583 (21.211)	data 0.001 (20.616)	eta 1 day, 23:32:55	loss 1.0645 (1.0645)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][20/404]	time 0.583 (10.898)	data 0.001 (10.309)	eta 1 day, 0:23:56	loss 1.0723 (1.0654)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][30/404]	time 0.581 (7.459)	data 0.001 (6.873)	eta 16:40:48	loss 1.0609 (1.0664)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][40/404]	time 0.583 (5.740)	data 0.000 (5.155)	eta 12:49:10	loss 1.0729 (1.0665)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][50/404]	time 0.579 (4.709)	data 0.001 (4.124)	eta 10:30:10	loss 1.0801 (1.0661)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][60/404]	time 0.583 (4.021)	data 0.001 (3.437)	eta 8:57:29	loss 1.0566 (1.0652)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][70/404]	time 0.580 (3.530)	data 0.000 (2.946)	eta 7:51:14	loss 1.0809 (1.0665)	acc 100.0000 (100.0000)	lr 0.000000
epoch: [131/150][80/404]	time 0.583 (3.162)	data 0.001

## AMS Grad optimiser. LR = 0.0015

In [ ]:
!python scripts/main.py --config-file configs/im_osnet_x1_0_softmax_256x128_amsgrad_cosine.yaml --transforms random_flip random_erase --root "/content/drive/MyDrive/OSNet/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15" data.save_dir log/osnet_x1_0_dukemtmcreid_softmax_cosinelr_90

Show configuration
adam:
  beta1: 0.9
  beta2: 0.999
cuhk03:
  classic_split: False
  labeled_images: False
  use_metric_cuhk03: False
data:
  combineall: False
  height: 256
  k_tfm: 1
  load_train_targets: False
  norm_mean: [0.485, 0.456, 0.406]
  norm_std: [0.229, 0.224, 0.225]
  root: /content/drive/MyDrive/OSNet/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15
  save_dir: log/osnet_x1_0_dukemtmcreid_softmax_cosinelr_90
  sources: ['market1501']
  split_id: 0
  targets: ['market1501']
  transforms: ['random_flip', 'random_erase']
  type: image
  width: 128
  workers: 4
loss:
  name: softmax
  softmax:
    label_smooth: True
  triplet:
    margin: 0.3
    weight_t: 1.0
    weight_x: 0.0
market1501:
  use_500k_distractors: False
model:
  load_weights: 
  name: osnet_x1_0
  pretrained: True
  resume: /content/drive/MyDrive/OSNet/deep-person-reid/log/osnet_x1_0_dukemtmcreid_softmax_cosinelr/model/model.pth.tar-50
rmsprop:
  alpha: 0.99
sampler:
  num_cams: 1
  num_datasets: